In [1]:
import pandas as pd
import numpy as np

In [2]:
df_1 = pd.read_csv('../_data/haile_data_raw.csv', engine='python', index_col=0).fillna(0)

In [3]:
df_2 = df_1[(df_1.bid_value_1 > 0.1) 
            & (df_1.bid_value_2 > 0.1)
             & (df_1.sale_method == 'S')].copy()

In [4]:
df_3 = df_2

In [5]:
df_3['adv_value'] = 0
for i in range(1,13):
    df_3.adv_value += df_3['volume_'+str(i)]*df_3['advertised_rate_'+str(i)]

In [6]:
df_3.volume_total_1 = df_3.volume_total_1.astype(float)

df_3 = df_3[df_3.adv_value > 0]
df_3 = df_3[df_3.volume_total_1 > 0]
df_3 = df_3[df_3.advertised_rate_1 > 0]

df_4 = df_3.copy()

In [7]:
df_4['ssq'] = df_4[['volume_'+str(i) for i in range(1,13)]].apply(np.square).sum(axis = 1)
df_4['sqs'] = df_4[['volume_'+str(i) for i in range(1,13)]].sum(axis = 1).apply(np.square)
df_4['hhi'] = df_4.ssq/df_4.sqs

In [8]:
df_4['auctionid'] = range(len(df_4))

In [9]:
common = ['auctionid', 'hhi', 'forest', 'adv_value', 'year', 'state', 'volume_total_1']
bidvalues = ['bid_value_'+str(i) for i in range(1,10)]

In [10]:
df_5 = df_4[common + bidvalues].copy()

In [11]:
df_6 = pd.DataFrame(np.vstack((
          df_5[common + ['bid_value_1']].values, 
          df_5[common + ['bid_value_2']].values, 
          df_5[common + ['bid_value_3']].values,
          df_5[common + ['bid_value_4']].values, 
          df_5[common + ['bid_value_5']].values, 
          df_5[common + ['bid_value_6']].values,
          df_5[common + ['bid_value_7']].values, 
          df_5[common + ['bid_value_8']].values, 
          df_5[common + ['bid_value_9']].values
)), columns = common +['actual_bid'])

In [12]:
df_6 = df_6[df_6.actual_bid > 1].copy()
df_6['auctionid'] = df_6.auctionid.astype(int)
df_6['forest'] = df_6.forest.astype(int)
df_6['state'] = df_6.state.astype(int)
df_6['year'] = df_6.year.astype(int)
len(df_6)

60758

In [13]:
df_6.reset_index(drop=True, inplace = True)
df_6

,auctionid,hhi,forest,adv_value,year,state,volume_total_1,actual_bid
0,0,0.469303,3,757400.0,82,30,620.0,3648800.0
1,1,0.668490,3,17839040.0,82,30,2590.0,21959040.0
2,2,0.732514,3,266000.0,82,30,460.0,1354560.0
3,3,0.593864,3,813790.0,82,30,370.0,1903830.0
4,4,0.797551,3,3115010.0,82,30,350.0,4169010.0
...,...,...,...,...,...,...,...,...
60753,15840,0.656042,6,10534613.0,79,22,620.0,14059693.0
60754,15842,0.697995,6,20905282.0,79,22,1051.0,21899811.0
60755,16046,0.507085,9,42937692.0,79,5,2938.0,44700492.0
60756,16183,0.705996,13,35221327.0,79,48,2535.0,37768496.0


In [14]:
df_6.to_csv('../_data/haile_data_prepared.csv')